In [ ]:
using Pkg; Pkg.activate(joinpath(@__DIR__, ".."))
using LinearAlgebra
using HybridRobotDynamics
using HiLQR

In [ ]:
"""
Solver Setup
"""

# Define bouncing quadrotor model
# Crazyflie 2.1 parameters: https://arxiv.org/pdf/1608.05786
e = 1.0
g = 9.81
m = 0.027
j = [1.436e-5, 1.395e-5,  2.173e-5]
b = 7.9379e-12 / 3.1582e-10 * ones(4)
c = 0.0283 * ones(4)
d = 0.0283 * ones(4)
system = bouncing_quadrotor(e, g, m, j, b, c, d)

# Stage and terminal costs
Q = 1e-6 * diagm([2e2*ones(3); 1e2*ones(4); ones(6)])
R = 1e-4 * I(system.nu)
Qf = 1e2 * Q
stage(x, u) = x'*Q*x + u'*R*u
terminal(x) = x'*Qf*x

# RK4 integrator
rk4 = ExplicitIntegrator(:rk4)

# Problem parameters
N = 50
Δt = 0.025
params = HiLQR.Parameters(system, stage, terminal, rk4, N, Δt)

# Reference trajectory
pref = [3.0, 3.0, 2.0]
qref = [1.0; zeros(3)]
xref = [pref; qref; zeros(6)]
uref = zeros(system.nu)
params.xrefs = [xref for k = 1:N]
params.urefs = [uref for k = 1:(N-1)]
us = [1e-1 * (rand(system.nu) - 0.5*ones(system.nu)) for k = 1:(N-1)]

# Initial conditions
p0 = [0.0, 0.0, 2.0]
q0 = [1.0, 0.0, 0.0, 0.0] #random_unit_quat()
v0 = 2e-0 * (rand(3) .- 0.5)
ω0 = zeros(3) #1e-1 * (rand(3) .- 0.5)
params.x0 = [p0; q0; v0; ω0]
params.mI = :flight

nothing

In [ ]:
"""
Solve using single-shooting HiLQR
"""

# Solve
sol = HiLQR.Solution(params)
sol.us .= us

# Solve
cache = HiLQR.Cache(params)
@time HiLQR.solve!(
    sol, cache, params;
    multishoot=false, max_step=.95, stat_tol=1e-6, max_iter=100
)

# Visualize states
xs = reduce(vcat, sol.xs)
plot_2d_states(N, system.nx, (1,3), xs; xlim=(-0.1,4), ylim=(-0.1,4), ylabel="z", title="Single Shooting")

f̃norms = [[norm(f̃)] for f̃ = sol.f̃s]
for k = 1:N-1
    f̃norms[k] = [k; f̃norms[k]]
end
f̃norms = reduce(vcat, f̃norms)
plot_2d_states(N-1, 2, (1,2), f̃norms; xlim=(0,N), ylim=(-0.01, 0.15), xlabel="time step", ylabel="dynamics defect norm", title="Intermediate Dynamics Defects")

In [ ]:
"""
Solve using multiple-shooting HiLQR
"""

# Solve
sol = HiLQR.Solution(params)
sol.us .= us
for k in 5:5:45
    sol.xs[k] = [xref[1:2]; -1e-3; xref[4:13]]
end

# Solve
cache = HiLQR.Cache(params)
@time HiLQR.solve!(
    sol, cache, params;
    multishoot=true, max_step=.95, stat_tol=1e-6, max_iter=100
)

# Visualize states
xs = reduce(vcat, sol.xs)
plot_2d_states(N, system.nx, (1,3), xs; xlim=(-0.1,4), ylim=(-0.1,4), ylabel="z", title="Multiple Shooting w/ Infeasible Guess")

f̃norms = [[norm(f̃)] for f̃ = sol.f̃s]
for k = 1:N-1
    f̃norms[k] = [k; f̃norms[k]]
end
f̃norms = reduce(vcat, f̃norms)
plot_2d_states(N-1, 2, (1,2), f̃norms; xlim=(0,N), ylim=(-0.01, 0.15), xlabel="time step", ylabel="dynamics defect norm", title="Intermediate Dynamics Defects")